# word2vec implementation

## Overview
<img src="image/word2vec_cbow_mechanism.png" align="left"/>

---
# Setups

In [1]:
import sys
import os
import re
from itertools import islice
from typing import Dict, List
import numpy as np
import tensorflow as tf

np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=400) 

## Setup for Google Colab environment

In [2]:
try:
    import google.colab
    IN_GOOGLE_COLAB = True
except:
    IN_GOOGLE_COLAB = False
    
if IN_GOOGLE_COLAB:
    !pip install line_profiler
    !google.colab.drive.mount('/content/gdrive')
    !rm -rf /content/github
    !mkdir -p /content/github
    !git clone https://github.com/oonisim/python-programs.git /content/github
        
    import sys
    sys.path.append('/content/github/nlp/src')

## Jupyter notebook setups

In [3]:
%load_ext line_profiler
%load_ext autoreload

## Utilites

In [4]:
%autoreload 2

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

import function.fileio as fileio

# Constants

In [5]:
USE_PTB = True
DEBUG = False
VALIDATION = True

---

# Data
## Corpus

In [6]:
corpus = "To be, or not to be, that is the question that matters"
_file = "ptb.train.txt"
if USE_PTB:
    if not fileio.Function.is_file(f"~/.keras/datasets/{_file}"):
        path_to_ptb = tf.keras.utils.get_file(
            _file, 
            f'https://raw.githubusercontent.com/tomsercu/lstm/master/data/{_file}'
        )
    corpus = fileio.Function.read_file(path_to_ptb)

In [7]:
examples = corpus.split('\n')[:5]
for line in examples:
    print(line)

 aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter 
 pierre <unk> N years old will join the board as a nonexecutive director nov. N 
 mr. <unk> is chairman of <unk> n.v. the dutch publishing group 
 rudolph <unk> N years old and former chairman of consolidated gold fields plc was named a nonexecutive director of this british industrial conglomerate 
 a form of asbestos once used to make kent cigarette filters has caused a high percentage of cancer deaths among a group of workers exposed to it more than N years ago researchers reported 


---
# Word indexing

In [8]:
%autoreload 2
from layer.preprocessing import (
    EventIndexing, 
    EventContext
)

## EventIndexing instance for the corpus

Adapt to the ```corpus``` and provides:
* event_to_index dictionary
* vocaburary of the corpus
* word occurrence probabilites

In [9]:
word_indexing = EventIndexing(
    name="word_indexing_on_ptb",
    corpus=corpus
)

In [10]:
words = word_indexing.list_events(range(10))
print(f"EventIndexing.vocabulary[10]:\n{words}\n")

indices = word_indexing.list_event_indices(words)
print(f"EventIndexing.event_to_index[10]:")
for item in zip(words, indices):
    print(item)

probabilities = word_indexing.list_probabilities(words)
print(f"\nEventIndexing.probabilities[10]:")
for word, p in zip(words, probabilities):
    print(f"{word:20s} : {p:.5e}")

EventIndexing.vocabulary[10]:
['<nil>' '<unk>' 'finance' 'targeting' 'perfect' 'commissioned' 'rainbow' 'redemptions' 'adjusted' 'amendments']

EventIndexing.event_to_index[10]:
('<nil>', 0)
('<unk>', 1)
('finance', 2)
('targeting', 3)
('perfect', 4)
('commissioned', 5)
('rainbow', 6)
('redemptions', 7)
('adjusted', 8)
('amendments', 9)

EventIndexing.probabilities[10]:
<nil>                : 0.00000e+00
<unk>                : 1.65308e-02
finance              : 2.72638e-04
targeting            : 3.87112e-05
perfect              : 4.07670e-05
commissioned         : 2.30178e-05
rainbow              : 2.05047e-05
redemptions          : 2.05047e-05
adjusted             : 1.32202e-04
amendments           : 3.66183e-05


## Sentence to Sequence

In [11]:
# sentences = "\n".join(corpus.split('\n')[5:6])
sentences = """
the asbestos fiber <unk> is unusually <unk> once it enters the <unk> 
with even brief exposures to it causing symptoms that show up decades later researchers said
"""
sequences = word_indexing.function(sentences)
for pair in zip(sentences.strip().split(" "), sequences[0]):
    print(f"{pair[0]:15} : {pair[1]:5}")

Sentence is empty. Skipping...
Sentence is empty. Skipping...


the             :  3302
asbestos        :  6850
fiber           :  4331
<unk>           :     1
is              :  5084
unusually       :  6782
<unk>           :     1
once            :  1675
it              :  7612
enters          :  9156
the             :  3302
<unk>           :     1

with           :     0
even            :     0
brief           :     0


---
# Context of a word in a sentence

In the sentence ```"a form of asbestos once used to make kent cigarette filters"```, one of the context windows ```a form of asbestos once``` of size 5 and event size 1 has.
* ```of``` as a target word.
* ```(a, form) and (asbestos, once)``` as its context.

### Sequence of the word indices for the sentence

In [12]:
sentences = """
a form of asbestos once used to make kent cigarette filters

N years old and former chairman of consolidated gold fields plc was named a nonexecutive director
"""

sequence = word_indexing.function(sentences)
sequence

Sentence is empty. Skipping...
Sentence is empty. Skipping...
Sentence is empty. Skipping...


<tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[ 971, 8320, 3254, 6850, 1675, 8484, 9072, 7580, 4410, 7564, 2809,    0,    0,    0,    0,    0],
       [1313, 6534, 8814, 8264, 3443, 2613, 3254, 3470, 5798,  518, 1849,  629, 4143,  971, 4048,  885]], dtype=int32)>

## Target, context pairs

For each word in the setence ```(of, asbestos, ... , kent)``` excludnig the ends of the sentence, create ```(target, context)``` as:

```
[
  [of, a, form, asbestos, once],              # target is 'of', context is (a, form, asbestos, once)
  ['asbestos', 'form', 'of', 'once', 'used'],
  ['once', 'of', 'asbestos', 'used', 'to'],
  ...
]
```

In [13]:
event_context = EventContext(
    name="ev",
    window_size=5,
    event_size=1    
)

In [14]:
event_context_pairs = event_context.function(sequence)
event_context_pairs

array([[[3254,  971, 8320, 6850, 1675],
        [6850, 8320, 3254, 1675, 8484],
        [1675, 3254, 6850, 8484, 9072],
        [8484, 6850, 1675, 9072, 7580],
        [9072, 1675, 8484, 7580, 4410],
        [7580, 8484, 9072, 4410, 7564],
        [4410, 9072, 7580, 7564, 2809],
        [7564, 7580, 4410, 2809,    0],
        [2809, 4410, 7564,    0,    0],
        [   0, 7564, 2809,    0,    0],
        [   0, 2809,    0,    0,    0],
        [   0,    0,    0,    0,    0]],

       [[8814, 1313, 6534, 8264, 3443],
        [8264, 6534, 8814, 3443, 2613],
        [3443, 8814, 8264, 2613, 3254],
        [2613, 8264, 3443, 3254, 3470],
        [3254, 3443, 2613, 3470, 5798],
        [3470, 2613, 3254, 5798,  518],
        [5798, 3254, 3470,  518, 1849],
        [ 518, 3470, 5798, 1849,  629],
        [1849, 5798,  518,  629, 4143],
        [ 629,  518, 1849, 4143,  971],
        [4143, 1849,  629,  971, 4048],
        [ 971,  629, 4143, 4048,  885]]], dtype=int32)

### Target, context pairs in textual words

In [15]:
word_indexing.sequence_to_sentence(event_context_pairs)

[[['of', 'a', 'form', 'asbestos', 'once'],
  ['asbestos', 'form', 'of', 'once', 'used'],
  ['once', 'of', 'asbestos', 'used', 'to'],
  ['used', 'asbestos', 'once', 'to', 'make'],
  ['to', 'once', 'used', 'make', 'kent'],
  ['make', 'used', 'to', 'kent', 'cigarette'],
  ['kent', 'to', 'make', 'cigarette', 'filters'],
  ['cigarette', 'make', 'kent', 'filters', '<nil>'],
  ['filters', 'kent', 'cigarette', '<nil>', '<nil>'],
  ['<nil>', 'cigarette', 'filters', '<nil>', '<nil>'],
  ['<nil>', 'filters', '<nil>', '<nil>', '<nil>'],
  ['<nil>', '<nil>', '<nil>', '<nil>', '<nil>']],
 [['old', 'n', 'years', 'and', 'former'],
  ['and', 'years', 'old', 'former', 'chairman'],
  ['former', 'old', 'and', 'chairman', 'of'],
  ['chairman', 'and', 'former', 'of', 'consolidated'],
  ['of', 'former', 'chairman', 'consolidated', 'gold'],
  ['consolidated', 'chairman', 'of', 'gold', 'fields'],
  ['gold', 'of', 'consolidated', 'fields', 'plc'],
  ['fields', 'consolidated', 'gold', 'plc', 'was'],
  ['plc', 'g